Run the following pip install to access the streamlit library in google colab

In [1]:
!pip install streamlit -q

The main function code is as follows:

In [9]:
%%writefile app.py

import os
import tensorflow as tf
import IPython.display as display

import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (12, 12)

import numpy as np
import PIL.Image
import time
import functools
import io
import requests
from tensorflow.keras.preprocessing.image import save_img
import streamlit as st
import os.path
import time

class CFG:
    debug=False
    epochs=10 if (debug==False) else 1
    steps_per_epoch=100 if (debug==False) else 10
    contentimg="contentimage.jpg"
    styleimg="styleimage.jpg"
    finalimg="Generatedimage.jpg"


def TrainModel(content_path,style_path):

    class Layers:
      StyleLayers = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1']
      ContentLayers = ['block5_conv2']
      NoContentLayers = len(ContentLayers)
      NoStyleLayers = len(StyleLayers)

    def tensor_to_image(tensor):
        tensor = tensor*255
        tensor = np.array(tensor, dtype=np.uint8)
        if np.ndim(tensor)>3:
            assert tensor.shape[0] == 1
            tensor = tensor[0]
        return PIL.Image.fromarray(tensor)


    def load_img(path_to_img):
        max_dim = 512
        img = tf.io.read_file(path_to_img)
        img = tf.image.decode_image(img, channels=3)
        img = tf.image.convert_image_dtype(img, tf.float32)

        shape = tf.cast(tf.shape(img)[:-1], tf.float32)
        long_dim = max(shape)
        scale = max_dim / long_dim

        new_shape = tf.cast(shape * scale, tf.int32)

        img = tf.image.resize(img, new_shape)
        img = img[tf.newaxis, :]
        return img

    content_path = content_path
    style_path = style_path
    content_image = load_img(content_path)
    style_image = load_img(style_path)

    def vgg_layers(layers):
        vgg = tf.keras.applications.VGG19(include_top=False, weights='imagenet')
        vgg.trainable = False

        outputs = [vgg.get_layer(layer).output for layer in layers]

        model = tf.keras.Model([vgg.input], outputs)
        return model

    def GramMatrix(IniTensor):
        result = tf.linalg.einsum('bijc,bijd->bcd', IniTensor, IniTensor)
        shape = tf.shape(IniTensor)
        Total = tf.cast(shape[1]*shape[2], tf.float32)
        return (result/(Total))


    class StyleContentModel(tf.keras.models.Model):
        def __init__(self, style_layers, content_layers):
            super(StyleContentModel, self).__init__()
            self.vgg = vgg_layers(style_layers + content_layers)
            self.style_layers = style_layers
            self.content_layers = content_layers
            self.num_style_layers = len(style_layers)
            self.vgg.trainable = False

        def call(self, inputs):
            inputs = 255.0*inputs
            preprocessed = tf.keras.applications.vgg19.preprocess_input(inputs)
            outputs = self.vgg(preprocessed)
            style_outputs, content_outputs = (outputs[:self.num_style_layers], outputs[self.num_style_layers:])

            style_outputs = [GramMatrix(style_output) for style_output in style_outputs]

            content_dict = {content_name: value for content_name, value in zip(self.content_layers, content_outputs)}

            style_dict = {style_name: value for style_name, value in zip(self.style_layers, style_outputs)}

            return {'content': content_dict, 'style': style_dict}

    extractor = StyleContentModel(Layers.StyleLayers, Layers.ContentLayers)
    style_targets = extractor(style_image)['style']
    content_targets = extractor(content_image)['content']
    image = tf.Variable(content_image)

    def getfloat(image):
      return tf.clip_by_value(image, clip_value_min=0.0, clip_value_max=1.0)

    optimiser = tf.keras.optimizers.Adam(learning_rate=0.02, beta_1=0.99, epsilon=1e-1)

    style_weight=1e-2
    content_weight=1e4
    total_variation_weight=30

    def Loss(outputs):
        style_outputs = outputs['style']
        content_outputs = outputs['content']
        style_loss = tf.add_n([tf.reduce_mean((style_outputs[name]-style_targets[name])**2) for name in style_outputs.keys()])
        style_loss *= style_weight / Layers.NoStyleLayers

        content_loss = tf.add_n([tf.reduce_mean((content_outputs[name]-content_targets[name])**2) for name in content_outputs.keys()])
        content_loss *= content_weight / Layers.NoContentLayers
        return style_loss+content_loss

    @tf.function()
    def train_step(image):
        with tf.GradientTape() as tape:
            outputs = extractor(image)
            loss = Loss(outputs)

        grad = tape.gradient(loss, image)
        optimiser.apply_gradients([(grad, image)])
        image.assign(getfloat(image))

    n = CFG.epochs
    m = CFG.steps_per_epoch
    for i in range(n):
        for j in range(m):
            train_step(image)
    save_img('Generatedimage.jpg', tensor_to_image(image))

def save_uploadedfile(uploadedfile,name):
     with open(os.path.join(".",name),"wb") as f:
         f.write(uploadedfile.getbuffer())
     return st.success("File Uploaded")

st.write("# Neural Style Transfer")
st.write("#### make sure the file is in a readable format")

contentImage = st.file_uploader("Upload the Image")
styleImage = st.file_uploader("Upload the Style")
if contentImage is not None and styleImage is not None:
    image = PIL.Image.open(contentImage)
    design = PIL.Image.open(styleImage)
    st.image(image, caption='Content Image')
    st.image(design, caption='Style Image')
    save_uploadedfile(contentImage,"contentimage.jpg")
    save_uploadedfile(styleImage,"styleimage.jpg")
    st.button('Generate Image', on_click=TrainModel,args=(CFG.contentimg,CFG.styleimg))

while True:
        if os.path.isfile(CFG.finalimg) :
            st.success("Result: ")
            FinalImage=PIL.Image.open(CFG.finalimg)
            st.image(FinalImage, caption='Generated Image')
            break
        time.sleep(1)

Overwriting app.py


Run the following code to generate the IP used as a password for tunnel website

In [11]:
! wget -q -O - ipv4.icanhazip.com

34.34.57.13


Run the following code to generate the website, accessed with the link provided

In [10]:
!streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.34.57.13:8501

npx: installed 22 in 2.06s
your url is: https://loud-schools-dance.loca.lt
2024-06-20 17:01:43.500512: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-20 17:01:43.500565: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-20 17:01:43.501989: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-20 17:01:43.510081: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized